In [10]:
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-4o-mini")

In [11]:
from typing import Optional,List

from langchain_core.pydantic_v1 import BaseModel, Field

In [12]:



class Person(BaseModel):
    """
    Person model representing an individual's basic information.
    Attributes:
        country (Optional[str]): Name of the country person belongs to.
        lastname (Optional[str]): Last name of the person.
        firstname (Optional[str]): First name of the person.
    """
    
    country: Optional[str] = Field(None, description="Name of the country person belongs to")
    lastname: Optional[str] = Field(None, description="Last name of the person")
    firstname: Optional[str] = Field(None, description="First name of the person")

class Persons(BaseModel):
    """
    Persons model representing a list of persons.
    Attributes:
        persons (List[Person]): List of persons.
    """
    
    persons: List[Person] = Field([], description="List of persons")

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages([
    ("system",
     "You are expert extraction specialist. You have been hired by a company to extract information from given text. You have to extract the name, last name and first name of each person. If any attribute is not present return null for that attribute.",
     ),
    ("human","{text}"),
])
cb= prompt| chatbot.with_structured_output(schema=Person)

In [14]:
comment = "I absolutely love this product! It's been a game-changer for my daily routine. The quality is top-notch and the customer service is outstanding. I've recommended it to all my friends and family. - Sarah Johnson, USA"
cb.invoke({"text":comment})

Person(country='USA', lastname='Johnson', firstname='Sarah')

In [15]:
chain_list = prompt | chatbot.with_structured_output(schema=Persons)
text_input = """
Alice Johnson from Canada recently reviewed a book she loved. Meanwhile, Bob Smith from the USA shared his insights on the same book in a different review. Both reviews were very insightful.
"""

# Invoke the processing chain on the text
chain_list.invoke({"text": text_input})

Persons(persons=[Person(country='Canada', lastname='Johnson', firstname='Alice'), Person(country='USA', lastname='Smith', firstname='Bob')])